In [1]:
import tensorflow
import pandas as pd
import numpy as np
import time
from PIL import Image
import glob
import matplotlib.pyplot as plt
import os
%matplotlib inline

# Ajouter les imports nécessaires

In [2]:
classes = ["basket","eye","binoculars","rabbit","hand"]
train = pd.read_csv('/kaggle/input/quickdraw-data/train.csv')
valid = pd.read_csv('/kaggle/input/quickdraw-data/valid.csv')


In [3]:
if not os.path.isdir('/kaggle/imagestrain'):
  os.mkdir('/kaggle/imagestrain')
for dir in classes:
  if not os.path.isdir('/kaggle/imagestrain/{d}'.format(d=dir)):
    os.mkdir('/kaggle/imagestrain/{d}'.format(d=dir))

pathsTrain = train.relative_path.to_list()

for path in pathsTrain:
  os.popen('cp /kaggle/input/quickdraw-data/images/{p} /kaggle/imagestrain/{p}'.format(p=path))

In [4]:
if not os.path.isdir('/kaggle/imagesValid'):
  os.mkdir('/kaggle/imagesValid')
for dir in classes:
  if not os.path.isdir('/kaggle/imagesValid/{d}'.format(d=dir)):
    os.mkdir('/kaggle/imagesValid/{d}'.format(d=dir))

pathsValid = valid.relative_path.to_list()

for path in pathsValid:
  os.popen('cp /kaggle/input/quickdraw-data/images/{p} /kaggle/imagesValid/{p}'.format(p=path))

In [5]:
from tensorflow.keras.preprocessing import image_dataset_from_directory
# from tensorflow.keras.preprocessing.image import ImageDataGenerator


train_ds = image_dataset_from_directory(directory = '/kaggle/imagestrain', labels ='inferred',label_mode='int', color_mode = "grayscale", validation_split = 0.2, subset = "training", seed = 42, image_size = (28,28), batch_size = 128 )
validation_ds = image_dataset_from_directory(directory = '/kaggle/imagestrain', labels = 'inferred',label_mode='int', color_mode = "grayscale", validation_split = 0.2, subset = "validation", seed = 42, image_size = (28,28), batch_size = 128)

In [ ]:
plt.figure(figsize=(20,20))
for im, tr in train_ds.take(200):
  #  print(im.numpy().astype("uint8"))
#   i+=1
# print(i)
  
  for i in range(9):
    ax = plt.subplot(3,3 , i +1)
    plt.imshow(tensorflow.squeeze(im[i]))
 #   print(tr[i].numpy())
    plt.title(tr[i].numpy())
    plt.axis("off")

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models


from tensorflow.keras.layers import Input, Dense, Activation,Lambda,Flatten,Dropout
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import L1, L2

from tensorflow.keras.optimizers import Adam

In [ ]:
model = Sequential()

#here i build a new one

#rescaling and dataaugmentation
model.add(layers.Rescaling(1.0 / 255,input_shape=(28,28,1)))
model.add(layers.RandomFlip("horizontal_and_vertical"))
model.add(layers.RandomRotation(0.6))
model.add(layers.BatchNormalization(axis=1))

#CONV1
model.add(Conv2D(32, (5, 5), kernel_regularizer= L2(1e-4), padding = 'same'))
model.add(layers.Activation("relu"))
model.add(layers.BatchNormalization(axis=1))

#CONV2
model.add(Conv2D(32, (5, 5), kernel_regularizer= L2(1e-4), padding = 'same'))
model.add(layers.Activation("relu"))
model.add(layers.BatchNormalization(axis=1))

#POOL + DROP
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(rate = 0.2))

#CONV3
model.add(Conv2D(64, (5, 5), kernel_regularizer= L2(1e-4), padding = 'same'))
model.add(layers.Activation("relu"))
model.add(layers.BatchNormalization(axis=1))

#CONV4
model.add(Conv2D(64, (5, 5), kernel_regularizer= L2(1e-4), padding = 'same'))
model.add(layers.Activation("relu"))
model.add(layers.BatchNormalization(axis=1))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(rate = 0.2))

#CONV5
model.add(Conv2D(128, (2,2), kernel_regularizer= L2(1e-4), padding = 'same'))
model.add(layers.Activation("relu"))
model.add(layers.BatchNormalization(axis=1))

#CONV6
model.add(Conv2D(128, (2, 2), activation='relu',   kernel_regularizer= L2(1e-4), padding = 'same'))
model.add(layers.Activation("relu"))
model.add(layers.BatchNormalization(axis=1))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(rate = 0.2))

#CONV7
model.add(Conv2D(128, (2,2), kernel_regularizer= L2(1e-4), padding = 'same'))
model.add(layers.Activation("relu"))
model.add(layers.BatchNormalization(axis=1))


#CONV8
model.add(Conv2D(128, (2, 2), activation='relu',   kernel_regularizer= L2(1e-4), padding = 'same'))
model.add(layers.Activation("relu"))
model.add(layers.BatchNormalization(axis=1))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(rate = 0.2))

#CONV7
model.add(Conv2D(128, (2,2), kernel_regularizer= L2(1e-4), padding = 'same'))
model.add(layers.Activation("relu"))
model.add(layers.BatchNormalization(axis=1))


#CONV8
model.add(Conv2D(128, (2, 2), activation='relu',   kernel_regularizer= L2(1e-4), padding = 'same'))
model.add(layers.Activation("relu"))
model.add(layers.BatchNormalization(axis=1))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Conv2D(64, (2, 2), activation='relu'))
model.add(Flatten())
model.add(Dropout(rate = 0.3))
#model.add(layers.BatchNormalization())
#model.add(Dense(1024, activation='relu',kernel_regularizer= L2()))
#model.add(Dropout(rate = 0.3))

#model.add(layers.BatchNormalization())
# model.add(Dense(512, activation='relu'))
# model.add(Dropout(rate = 0.5))
#model.add(layers.BatchNormalization())
#model.add(Dense(128, activation='relu',kernel_regularizer= L2()))
#model.add(Dropout(rate = 0.75))

#model.add(layers.BatchNormalization())
#model.add(Dense(64, activation='relu',kernel_regularizer= L2()))
#model.add(Dropout(rate = 0.4))

model.add(layers.BatchNormalization())
model.add(Dense(5, activation='softmax'))

model.summary()


In [ ]:
from tensorflow.python.keras import callbacks

from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
opt = Adam(learning_rate=0.001)
checkpointer = ModelCheckpoint(filepath='model.weights.best.hdf5', verbose=1,
     save_best_only=True)
ca = EarlyStopping(patience =10,min_delta=0.001 , monitor = 'val_loss')
model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics = ['sparse_categorical_accuracy'])

In [ ]:
hist = model.fit(
    train_ds, epochs=100, validation_data=validation_ds, callbacks = [checkpointer, ca], batch_size=128
)

In [ ]:
history_df = pd.DataFrame(hist.history)
history_df.loc[:, ['loss', 'val_loss']].plot();
print("Minimum validation loss: {}".format(history_df['val_loss'].min()))

In [ ]:
test_ds = image_dataset_from_directory(directory = '/kaggle/imagesValid/', labels ='inferred',label_mode='int', color_mode = "grayscale", image_size = (28,28),shuffle=False)

In [ ]:
score = model.evaluate(test_ds,batch_size=128, verbose=1)


In [ ]:
x = list(range(1,61))
l= hist.history['val_loss']
ll= hist.history['val_sparse_categorical_accuracy']
plt.subplots(1,1)
plt.xlabel('epoch')
plt.ylabel('Entropie croisée')
plt.title("Test : Score = {0:5.3f}, Précision = {1:5.3f}".format(score[0], score[1]))
plt.plot(x,l)
plt.plot(x,ll)
plt.show()
plt.tight_layout()

In [ ]:
predicted_classes = np.argmax(model.predict(test_ds), axis=-1)

labels = valid.class_label.to_list()

incorrects = np.nonzero(predicted_classes != labels)[0]
print(incorrects)
plt.figure(figsize=(10,10))

for i in range(0,len(incorrects)):
    plt.subplot(3,3,i+1)
    idx = incorrects[i]
    im = Image.open('/kaggle/imagesValid/'+valid.relative_path[idx])
    plt.imshow(im)
    plt.title( "Prédit/vrai : {}/{}".format(predicted_classes[idx], labels[idx]))
    plt.xticks([])
    plt.yticks([])
plt.tight_layout()
print("Nombre d'erreurs {}/{}\n\n".format(len(incorrects),len(labels)))
len(incorrects)

In [ ]:
print(len(incorrects))
print(len(labels))
print(len(predicted_classes))

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(labels, predicted_classes )
plt.imshow(cm, cmap=plt.cm.Blues)
plt.colorbar()
indexes = np.arange(len(classes))
for i in indexes:
    for j in indexes:
        plt.text(j, i, cm[i, j])
plt.xticks(indexes, classes, rotation=90)
plt.xlabel('Predicted class')
plt.yticks(indexes, classes)
plt.ylabel('True class')
plt.title('Confusion matrix')
plt.show()

In [ ]:

from sklearn.metrics import confusion_matrix

cm = confusion_matrix(labels, predicted_classes )
plt.imshow(cm, cmap=plt.cm.Blues)
plt.colorbar()
indexes = np.arange(len(classes))
for i in indexes:
    for j in indexes:
        plt.text(j, i, cm[i, j])
plt.xticks(indexes, classes, rotation=90)
plt.xlabel('Predicted class')
plt.yticks(indexes, classes)
plt.ylabel('True class')
plt.title('Confusion matrix')
plt.show()

In [ ]:
model.save('/kaggle/poids.h5')


In [ ]:
mm = tensorflow.keras.models.load_model('/kaggle/poids.h5') 

In [ ]:
mm.summary()

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np

from PIL import Image

import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Input, Dense, Activation, Lambda, Flatten, Dropout, Conv2D, BatchNormalization, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.optimizers import Adam

from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras import callbacks

import pathlib
import os
from zipfile  import ZipFile

In [ ]:
def Create_Dataset(dataset_file_path, images_path):
  dataset = pd.read_csv(dataset_file_path)

  images_list = []

  for i in range(dataset.index.stop):
    image = Image.open( images_path + dataset.iloc[i,1])
    image = np.array(image, dtype=float).reshape(28,28,1)
    image /= 255.0
    images_list.append(image)

  X = np.asarray(images_list)
  Y = np.asarray(to_categorical(dataset['class_label'], 5))

  return X, Y

X_train, Y_train = Create_Dataset("/kaggle/input/quickdraw-data/train.csv", "/kaggle/input/quickdraw-data/images/")
X_valid, Y_valid = Create_Dataset("/kaggle/input/quickdraw-data/valid.csv", "/kaggle/input/quickdraw-data/images/")

In [ ]:
def Preprocess_Data(x_, y_, batch_size, seed):
  generator = ImageDataGenerator()
  data = generator.flow(
                        x=x_, 
                        y=y_, 
                        batch_size=batch_size, 
                        seed=seed
                        )
  return data

def Preprocess_Data_With_Augmentation(x_, y_, batch_size, seed):
  generator = ImageDataGenerator(
                                  featurewise_center=False,
                                  samplewise_center=False,
                                  featurewise_std_normalization=False,
                                  samplewise_std_normalization=False,
                                  zca_whitening=False,
                                  rotation_range=3,
                                  zoom_range = 0.1,
                                 
                                 )
  augmented_data = generator.flow(
                        x=x_, 
                        y=y_, 
                        batch_size=batch_size, 
                        seed=seed
                        )
  return augmented_data

In [ ]:
from tensorflow.keras.regularizers import L2
def Create_Model(dropout):
  model = Sequential()

  model.add(Conv2D(filters = 16 , kernel_size = 3, padding="same", activation='relu', input_shape=(28, 28, 1),kernel_regularizer= L2(1e-4) ))
  model.add(Conv2D(filters = 16 , kernel_size = 3, padding="same", activation='relu',kernel_regularizer= L2(1e-4) ))
  model.add(BatchNormalization())

  model.add(MaxPooling2D())
  model.add(Dropout(rate=dropout))
  model.add(Conv2D(filters = 32 , kernel_size = 3, padding="same", activation='relu',kernel_regularizer= L2(1e-4) ))
  model.add(Conv2D(filters = 32 , kernel_size = 3, padding="same", activation='relu',kernel_regularizer= L2(1e-4) ))
  model.add(BatchNormalization())
    
  model.add(MaxPooling2D())
  model.add(Dropout(rate=dropout))
  model.add(Conv2D(filters = 64 , kernel_size = 3, padding="same",  activation='relu',kernel_regularizer= L2(1e-4)))
  model.add(Conv2D(filters = 64 , kernel_size = 3, padding="same",  activation='relu',kernel_regularizer= L2(1e-4)))
  model.add(BatchNormalization())

  model.add(MaxPooling2D())
  model.add(Dropout(rate=dropout))
  model.add(Conv2D(filters = 128 , kernel_size = 2, padding="same",  activation='relu',kernel_regularizer= L2(1e-4)))
  model.add(Conv2D(filters = 128 , kernel_size = 2, padding="same",  activation='relu',kernel_regularizer= L2(1e-4)))

  model.add(BatchNormalization())
    
  model.add(GlobalAveragePooling2D())

  model.add(Dropout(rate=dropout))
  model.add(Flatten())
  model.add(Dense(128, activation='relu'))
  model.add(BatchNormalization())
  model.add(Dense(5, activation='softmax'))

  return model

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

def train_model(model, d_train, d_valid, learning_rate, batch_size, num_epochs):
  loss_fn = tf.keras.losses.CategoricalCrossentropy()
  optimizer = Adam(learning_rate=learning_rate)

  # Callback pour la sauvegarde du meilleur modèle
  if not os.path.isdir("sauve1"):
      os.mkdir("sauve1")
    
  checkpoint = ModelCheckpoint("sauve1/Model-loss-{val_loss:.2f}-accuracy-{val_accuracy:.2f}.h5",
                                save_best_only=True,verbose=1)

  model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

  early_stopping = callbacks.EarlyStopping(
      min_delta=0.001,
      patience=5,
      restore_best_weights=True,
  )

  # Entraînement
  history = model.fit(d_train,
                      validation_data=d_train,
                      batch_size = batch_size, 
                      epochs=num_epochs,
                      callbacks=[early_stopping, checkpoint],
                      verbose=1)
  
  return history

In [ ]:
D_train = Preprocess_Data_With_Augmentation(X_train, Y_train, 32, 1)
D_valid = Preprocess_Data(X_valid, Y_valid,32, 1)

model = Create_Model(0.5)

history = train_model(model, D_train, D_valid, 0.001, 32, 100)

In [ ]:
image_gen = ImageDataGenerator()
    
D_test = image_gen.flow(
    X_valid,
    Y_valid,
    batch_size=1,
    seed=1)

model.evaluate(D_test, verbose=1)

In [ ]:
history_frame = pd.DataFrame(history.history)

history_frame.loc[:, ['loss', 'val_loss']].plot()
history_frame.loc[:, ['accuracy', 'val_accuracy']].plot();

In [ ]:
predicted_classes = np.argmax(model.predict(X_valid), axis=-1)
y = np.argmax(Y_valid,axis=1)

incorrects = np.nonzero(predicted_classes != y)[0]

plt.figure(figsize=(10,10))
for i in range(0,9):
    plt.subplot(3,3,i+1)
    plt.imshow(X_valid[incorrects[i]].reshape(28,28), cmap='gray', interpolation='none')
    plt.title( "Prédit/vrai : {}/{}".format(predicted_classes[incorrects[i]], y[incorrects[i]]))
    plt.xticks([])
    plt.yticks([])
plt.tight_layout()

print("Nombre d'erreurs {}/{}\n\n".format(len(incorrects),len(Y_valid)))